# Source
https://thinkinfi.com/warp-perspective-opencv/

# Imports

In [ ]:
import configparser
import cv2  # OpenCV - change image
import math
import numpy as np

# Global Vars

In [ ]:
config = configparser.ConfigParser()
config.read("config.ini")
BASE_FP = config["FILES"]["default_path"]

# Functions

In [ ]:
def resize_with_aspect_ratio(image, width=None, height=None, inter=cv2.INTER_AREA):
    """This is better than our first pass of resize_window"""
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

In [ ]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
def clahe(img, clipLimit: float, tileGridSize):
    """Address glare and bright spots

    A bright photo will have all its pixels confined to a relatively high value. Picture a histogram that is thin and tall. CLAHE will essentially normalize it, spreading the values so the histogram is shorter and wider.
    """
    clahe_ = cv2.createCLAHE(clipLimit, tileGridSize)

    new_img = clahe_.apply(img)
    return new_img

In [ ]:
def find_corners(morph, original):
    """
    Thanks https://stackoverflow.com/questions/60941012/how-do-i-find-corners-of-a-paper-when-there-are-printed-corners-lines-on-paper-i
    Algorithm above
    1. Apply grayscale
    2. Apply blur
    3. Apply threshold (i.e. black & white)
    4. Apply morphology
    5. Find contours
    6. Approximate polygon -> 4 vertices that represent corners

    To allow flexibility, we'll skip steps 1-4. Main takeaway is to provide an image where, as best as possible, only 4 contours can be found. Typically will be a black & white image
    """

    # get largest contour
    contours = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    area_thresh = 0
    for c in contours:
        area = cv2.contourArea(c)
        if area > area_thresh:
            area_thresh = area
            big_contour = c

    # get perimeter and approximate a polygon
    peri = cv2.arcLength(big_contour, True)
    # note that approxPolyDP looks for any polygon, not specifically a rectangle
    corners = cv2.approxPolyDP(big_contour, 0.04 * peri, True)

    corners = [corner[0] for corner in corners]

    # Add blue circles onto image
    # for corner in corners:
    #     x, y = corner.ravel()
    #     cv2.circle(original, (x, y), 50, (255, 0, 0), -1)
    resize = resize_with_aspect_ratio(original, 300, 200)
    cv2.imshow("blue corners", resize)

    return corners

In [ ]:
def getAngle(a, b, c):
    # https://manivannan-ai.medium.com/find-the-angle-between-three-points-from-2d-using-python-348c513e2cd
    ang = math.degrees(
        math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0])
    )
    return ang + 360 if ang < 0 else ang

In [ ]:
def best_corners_angle(corners):
    best_corners = set()
    n = len(corners)

    for i in range(n):
        for j in range(i + 1, n):
            for k in range(j + 1, n):
                angle = getAngle(corners[i], corners[j], corners[k])

                if abs(angle - 90) < 2:
                    best_corners.add(tuple(corners[i]))
                    best_corners.add(tuple(corners[j]))
                    best_corners.add(tuple(corners[k]))

    if len(best_corners) != 4:
        print(f"After checking the angles, cannot find exactly 4 corners")
        return corners

    # return as list because sets are not ordered
    return list(best_corners)

In [ ]:
def order_points(vertices):
    # https://stackoverflow.com/questions/66916409/python-find-most-upper-left-coordinate-and-corners-in-opencv-numpy
    # order: [tl, tr, bl, br]
    sums = [sum(vertex) for vertex in vertices]
    tl_sum = min(sums)
    br_sum = max(sums)

    for vertex in vertices:
        sum_ = sum(vertex)
        if sum_ == tl_sum:
            tl = vertex
        elif sum_ == br_sum:
            br = vertex

    diffs = [(vertex[1] - vertex[0]) for vertex in vertices]
    tr_diff = min(diffs)
    bl_diff = max(diffs)

    for vertex in vertices:
        diff = vertex[1] - vertex[0]
        if diff == tr_diff:
            tr = vertex
        elif diff == bl_diff:
            bl = vertex

    print(f"ordered: {[tl, tr, bl, br]}")
    return [tl, tr, bl, br]

In [ ]:
def warp_perspective(img, corners):
    height, width, *_ = img.shape
    print(f"{width=}, {height=}")
    # Get top-left, top-right, bottom-left, bottom-right points
    # tl, bl, br, tr = corners
    tl, tr, bl, br = corners

    point_matrix = np.float32([tl, tr, bl, br])

    # cv2.circle(img, (tl[0], tl[1]), 50, (0, 0, 255), cv2.FILLED)
    # cv2.circle(img, (tr[0], tr[1]), 50, (0, 255, 0), cv2.FILLED)
    # cv2.circle(img, (bl[0], bl[1]), 50, (255, 0, 0), cv2.FILLED)
    # cv2.circle(img, (br[0], br[1]), 50, (0, 0, 0), cv2.FILLED)

    converted_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]])

    perspective_transform = cv2.getPerspectiveTransform(point_matrix, converted_points)

    warp_perspective = cv2.warpPerspective(img, perspective_transform, (width, height))

    resize = resize_with_aspect_ratio(img, 300, 200)
    cv2.imshow("warp perspective corners", resize)
    return warp_perspective

# Main

In [ ]:
apply_gray = False
apply_clahe = True

# BASE_FP ..\static\
fp = BASE_FP + "boiling-crab.jpg"

In [ ]:
event_name = fp.split("\\")[-1].split(".")[0]
event_name

In [ ]:
imgs = {}

In [ ]:
if apply_gray:
    ...

elif apply_clahe:
    original = cv2.imread(fp)
    imgs["original"] = original

    gray = grayscale(original)
    imgs["gray"] = gray

    # clahe() needs a grayscale img
    clahe_ = clahe(gray, 8.0, (8, 8))
    imgs["clahe"] = clahe_

    blur = cv2.GaussianBlur(clahe_, (51,51), 0)
    imgs["blur"] = blur

    _, bw = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    imgs["bw"] = bw

    kernel = np.ones((7, 7), np.uint8)
    morph = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)
    imgs["morphology"] = morph

    corners = find_corners(morph, imgs["original"])
    print(corners)
    if len(corners) > 4:
        corners = best_corners_angle(corners)
        print(f"corners after angle: {corners}")
    elif len(corners) < 4:
        print(f"Found less than 3 corners")
    corners = order_points(corners)

    warp = warp_perspective(original, corners)
    imgs["warp"] = warp

In [ ]:
# Display all imgs into separate windows
for name, img in imgs.items():
    # if "warp" in name:
    #     resize = resize_window(img, 600, 800, "warp")
    #     cv2.imshow(name, resize)
    # else:
    #     resize = resize_with_aspect_ratio(img, 300, 200)
    #     cv2.imshow(name, resize)
    print(f"{name}: {img.shape}")
    resize = resize_with_aspect_ratio(img, 400, 200)
    cv2.imshow(name, resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imwrite(f"static/changes/{event_name}.warped.JPG", warp)